In [34]:
# carga la extensión autoreload
%load_ext autoreload

# establece el nivel de autorecarga en 2
%autoreload 2

import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import pandas as pd
from db.client import MongoDBConnection

### CONEXIÓN A LA BASE DE DATOS

In [36]:
mongo_connection = MongoDBConnection()
db = mongo_connection.get_database()

# CALCULO TMI

In [37]:
df = pd.DataFrame(list(db['tmi_valores_mensuales'].find({}, {'_id': 0})))
datos_n = pd.DataFrame(list(db['n'].find({}, {'_id': 0})))

In [38]:
from services.calculos_tmi import calcular_indices

In [64]:
calcular_indices(datos_base=df, datos_n=datos_n, grados=2, minutos=26, segundos=0, orientacion='norte', amax=100)

LATITUD: 2.5


(           mes  temperatura  precipitacion   d         i      ETPsc     N  \
 0        enero        18.49         192.15  31  7.242723  67.362541  11.9   
 1      febrero        18.35         156.41  28  7.159858  66.384493  11.9   
 2        marzo        18.51         168.63  31  7.254588  67.502823  12.0   
 3        abril        18.55         178.27  30  7.278336  67.783807  12.1   
 4         mayo        18.58         169.66  31  7.296165  67.994913  12.1   
 5        junio        18.73          64.58  30  7.385529  69.055172  12.1   
 6        julio        18.92          56.99  31  7.499253  70.409475  12.1   
 7       agosto        19.42          46.95  31  7.801332  74.033754  12.1   
 8   septiembre        19.21          60.26  30  7.673966  72.500915  12.0   
 9      octubre        18.23         208.70  31  7.089089  65.551637  11.9   
 10   noviembre        18.07         339.42  30  6.995102  64.449021  11.9   
 11   diciembre        17.95         280.48  31  6.924892  63.62

In [40]:
P

1922.5000000000002

In [41]:
df

,mes,temperatura,precipitacion,d
0,enero,18.49,192.15,31
1,febrero,18.35,156.41,28
2,marzo,18.51,168.63,31
3,abril,18.55,178.27,30
4,mayo,18.58,169.66,31
5,junio,18.73,64.58,30
6,julio,18.92,56.99,31
7,agosto,19.42,46.95,31
8,septiembre,19.21,60.26,30
9,octubre,18.23,208.70,31


In [42]:
# Datos
grados = 2
minutos = 26
segundos = 0

latitud_decimal_corrected = grados + minutos / 60 + segundos / 3600
LATITUD = round(latitud_decimal_corrected * 2) / 2 # F7
ORIENTACION = 'norte' # E7
AMAX = 100 # J8

In [43]:
df['i'] = (df['temperatura']/5) ** 1.514

In [44]:
P = df['precipitacion'].sum()
I = df['i'].sum()
a = (0.000000675 * I**3) - (0.0000771 * I**2) + (0.01792 * I) + 0.49239

In [45]:
df['ETPsc'] = 16 * (10 * df['temperatura'] / I) ** a

In [46]:
# Consultar los datos N (dias con sol por mes)
datos_n = pd.DataFrame(list(db['n'].find({}, {'_id': 0})))

In [47]:
datos_n_select = datos_n[(datos_n['orientacion']==ORIENTACION)&(datos_n['latitud']==LATITUD)]

In [48]:
df['N'] = datos_n_select[df['mes'].to_list()].values[0] 

In [49]:
df['ETPcrr'] = df['ETPsc']*df['N']/12*df['d']/30

In [50]:
ETP = df['ETPcrr'].sum()

In [51]:
# Crear una lista para almacenar los valores calculados de 'balance_integridad_asfalto'
balance_integridad_asfalto = []

# Inicializar el valor inicial de Ai
Ai_anterior = AMAX

# Iterar sobre cada fila y calcular balance_integridad_asfalto basado en el valor de Ai anterior
for index, row in df.iterrows():
    if index == 0:
        # Primer cálculo, usa AMAX
        balance = AMAX + row['precipitacion'] - row['ETPcrr']
    else:
        # En los cálculos siguientes, usa el valor de Ai anterior
        balance = Ai_anterior + row['precipitacion'] - row['ETPcrr']
    
    # Guardar el valor calculado de balance_integridad_asfalto
    balance_integridad_asfalto.append(balance)
    
    # Calcular Ai y actualizar Ai_anterior para la siguiente iteración
    Ai_anterior = max(0, min(100, balance))

df['balance_integridad_asfalto'] = balance_integridad_asfalto

Ai_values = [max(0, min(100, balance)) for balance in balance_integridad_asfalto]
df['Ai'] = Ai_values

In [52]:
df['EXC'] = df['balance_integridad_asfalto'].apply(
    lambda M: 0 if M <= 100 else (M - 100 if M > 100 else "Error")
)

In [53]:
EXC = df['EXC'].sum()

In [54]:
df['DEF'] = df['balance_integridad_asfalto'].apply(
    lambda M: 0 if M >= 0 else (abs(M) if M < 0 else "Error")
)

In [55]:
DEF = df['DEF'].sum()

In [56]:
# Índice de humedad (Ih)
Ih = (EXC / ETP) * 100 if ETP > 0 else 0

# Índice de aridez (Ia)
Ia = (DEF / ETP) * 100 if ETP > 0 else 0

# Índice de Thornthwaite (Im)
Im = Ih - 0.6 * Ia

# Índice de Thornthwaite 1955 (Im)
Im_1955 = 100 * (P / ETP - 1) if ETP > 0 else 0

# Cálculo del Índice de Thornthwaite, Witzack
Im_Witzack = 75*((P/ETP)-1)+10

# Resultados
resultados = {
    'Índice de humedad (Ih)': Ih,
    'Índice de aridez (Ia)': Ia,
    'Índice de Thornthwaite (Im)': Im,
    'Índice de Thornthwaite 1955': Im_1955,
    'Índice de Thornthwaite, Witzack': Im_Witzack
}

resultados

{'Índice de humedad (Ih)': 132.0914011135646,
 'Índice de aridez (Ia)': 0.0,
 'Índice de Thornthwaite (Im)': 132.0914011135646,
 'Índice de Thornthwaite 1955': 132.0914011135646,
 'Índice de Thornthwaite, Witzack': 109.06855083517344}

In [57]:
# Aplicar la lógica de la fórmula para la clasificación climática
if Im <= -40:
    clasificacion_climatica = "E: Árido"
elif -40 < Im <= -20:
    clasificacion_climatica = "D: Semi-árido"
elif -20 < Im <= 0:
    clasificacion_climatica = "C1: Subhúmedo-seco"
elif 0 < Im <= 20:
    clasificacion_climatica = "C2: Subhúmedo-húmedo"
elif 20 < Im <= 40:
    clasificacion_climatica = "B1: Ligeramente húmedo"
elif 40 < Im <= 60:
    clasificacion_climatica = "B2: Moderadamente húmedo"
elif 60 < Im <= 80:
    clasificacion_climatica = "B3: Húmedo"
elif 80 < Im <= 100:
    clasificacion_climatica = "B4: Muy húmedo"
else:
    clasificacion_climatica = "A: Excesivamente húmedo"

In [58]:
resultados ['Clasificación climática'] = clasificacion_climatica

In [59]:
resultados

{'Índice de humedad (Ih)': 132.0914011135646,
 'Índice de aridez (Ia)': 0.0,
 'Índice de Thornthwaite (Im)': 132.0914011135646,
 'Índice de Thornthwaite 1955': 132.0914011135646,
 'Índice de Thornthwaite, Witzack': 109.06855083517344,
 'Clasificación climática': 'A: Excesivamente húmedo'}

In [60]:
df

,mes,temperatura,precipitacion,d,i,ETPsc,N,ETPcrr,balance_integridad_asfalto,Ai,EXC,DEF
0,enero,18.49,192.15,31,7.242723,67.362541,11.9,69.027893,223.122107,100.000000,123.122107,0
1,febrero,18.35,156.41,28,7.159858,66.384493,11.9,61.442537,194.967463,100.000000,94.967463,0
2,marzo,18.51,168.63,31,7.254588,67.502823,12.0,69.752917,198.877083,100.000000,98.877083,0
3,abril,18.55,178.27,30,7.278336,67.783807,12.1,68.348672,209.921328,100.000000,109.921328,0
4,mayo,18.58,169.66,31,7.296165,67.994913,12.1,70.846922,198.813078,100.000000,98.813078,0
5,junio,18.73,64.58,30,7.385529,69.055172,12.1,69.630632,94.949368,94.949368,0.000000,0
6,julio,18.92,56.99,31,7.499253,70.409475,12.1,73.362762,78.576606,78.576606,0.000000,0
7,agosto,19.42,46.95,31,7.801332,74.033754,12.1,77.139059,48.387548,48.387548,0.000000,0
8,septiembre,19.21,60.26,30,7.673966,72.500915,12.0,72.500915,36.146633,36.146633,0.000000,0
9,octubre,18.23,208.70,31,7.089089,65.551637,11.9,67.172219,177.674414,100.000000,77.674414,0
